# Flaw Engine: EMERGENCY FIX Cloud Trainer ♟️🚀

**IF GITHUB SYNC IS FAILING**: Copy the code from the notification and paste it into these cells manually.

### 🏁 Step 1: Clean Clone
Wipes everything and clones fresh.

In [ ]:
import os
import shutil

GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"
TARGET_FOLDER = "/content/flaw_worker"

print("Starting Clean Slate...")
!rm -rf {TARGET_FOLDER}
!git clone {GITHUB_URL} {TARGET_FOLDER}

if os.path.exists(TARGET_FOLDER):
    print("\n✅ SUCCESS! Folder found.")
    %cd {TARGET_FOLDER}
    !ls -F
else:
    print("\n❌ FAILED. Is your URL correct and Repo Public?")

### 🏁 Step 2: Brute Force Compile
Compiles the engine for Linux.

In [ ]:
import os
import shutil
import glob

print("Installing dependencies...")
!apt install -y cmake g++ > /dev/null
!pip install requests torch > /dev/null

TARGET_FOLDER = "/content/flaw_worker"
# Auto-detect if there is a nested folder (e.g. flaw_worker/flaw/core)
root = TARGET_FOLDER
if os.path.exists(os.path.join(TARGET_FOLDER, 'flaw', 'core')):
    root = os.path.join(TARGET_FOLDER, 'flaw')

CORE_DIR = os.path.join(root, "core")
BUILD_DIR = os.path.join(CORE_PATH, "build") # wait, fixed below
BUILD_DIR = os.path.join(CORE_DIR, "build")

print(f"Detected Root: {root}")

if os.path.exists(BUILD_DIR): shutil.rmtree(BUILD_DIR)
os.makedirs(BUILD_DIR, exist_ok=True)

%cd {BUILD_DIR}
!cmake ..
!make -j$(nproc)

print("Deploying...")
libs = glob.glob("flaw_core*.so")
if libs:
    lib = os.path.abspath(libs[0])
    shutil.copy2(lib, os.path.join(root, "flaw_core.so"))
    shutil.copy2(lib, os.path.join(root, "ai", "flaw_core.so"))
    print("✅ Core Deployed!")
else:
    print("❌ Build FAILED. See output above.")

### 🏁 Step 3: Launch Training

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

import sys
import os

# Ensure the root we detected is in the path
if root not in sys.path: sys.path.insert(0, root)

try:
    import flaw_core
    from ai.selfplay_worker import DistributedWorker
    print("🚀 Starting...")
    worker = DistributedWorker(MASTER_URL, depth=1)
    worker.run()
except Exception as e:
    print(f"❌ ERROR: {e}")